# Reinforcement Learning Tarea 1 - K Armed Bandits

## Objetivos
- Demostrar entendimiento sobre K-Armed Bandits
- Implementar varios algoritmos vistos en el curso teórico 
- comparar su performance en un mismo problema (ambiente).

## A entregar
- Implementación de agente arbitrario.
- Implementación de agente epsilon-greedy.
- Implementación de agente epsilon-greedy optimista.
- Implementación de agente upper confidence bound.
- Grafique la evolución de las recompensas (reward) a lo largo del tiempo para cada algoritmo, promediado sobre 2000 ejecuciones.
- Escriba un reporte sobre sus implementaciones comparando los distintos agentes implementados.
- [Extra] Implementación de epsilon-greedy con decay.

## Descripción del ambiente a usar

Vamos a usar KBanditsEnv, un ambiente implementado con [Gymnasium](https://gymnasium.farama.org/index.html) pero orientada a [Armed Bandits](https://en.wikipedia.org/wiki/Multi-armed_bandit). 

Recordando el teórico, sabemos que un K Armed Bandit representa una maquina tragamonedas con multiples brazos (K) y donde nuestra recompensa depende de cual brazo tiremos.

En este caso particular, `k_bandits_env/KBanditsGaussian-v0` es el ambiente a utilizar (página 28 del libro de Sutton y Barto), cuenta con 10 brazos (K=10) y por cada interaccion con el ambiente (`step`) nustra recompensa (reward) depende de que brazo elegimos entre los 10 posibles. En este ambiente, todos los brazos tienen una recompensa (que puede ser negativa en algunos casos) y nuestro objetivo es diseñar agentes que maximicen la suma total de recompensas a lo largo de 1000 interacciones con el ambiente (`1000 steps`).

![KBandits](https://miro.medium.com/v2/resize:fit:720/format:webp/1*iguhq3SaQd730c-TYI8QZA.png)

El valor verdadero ($q_*(a)$) de cada brazo es muestreado de una distribución normal con media 0 y varianza 1. La recompensa de cada brazo es muestreada de una distribución normal con media $q_*(a)$ y varianza 1.


> Nota: El agente no tiene acceso a los valores verdaderos de los brazos, solo puede interactuar con el ambiente, es decir, seleccionar un brazo y recibir una recompensa. Nosotros usaremos los valores verdaderos para comparar la performance de los distintos agentes.

Algunas preguntas:
- ¿Es un ambiente estacionario o no estacionario?
- ¿Qué se espera de un agente que seleccione brazos de manera aleatoria en terminos de recompensa y % de veces que selecciona el mejor brazo?

## Definición del ambiente e imports necesarios

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random

from Agent import Agent
from Utils import get_env, plot_agents_performance

Algunas constantes que vamos a usar:

In [ ]:
NUM_STEPS = 1000
NUM_EPISODES = 2000

In [ ]:
# creamos un entorno de ejemplo, con print_true_values podemos ver los valores verdaderos de las acciones
# PD: en teoria no deberiamos tener acceso a estos valores pero nos sirve para comparar
sample_env = get_env(print_true_values=True) 
print(sample_env.action_space)

## Implementación de Agentes

En esta sección, exploraremos cómo crear agentes concretos heredando de la clase abstracta `Agent`. Esta clase base define la estructura y el comportamiento general que cualquier agente debe seguir al interactuar con un entorno, proporcionando un marco consistente para diversas estrategias de toma de decisiones.


**Pasos para Implementar un Agente Concreto**

1. **Definir el Constructor (`__init__`)**: Inicializa los atributos específicos del agente, incluyendo su nombre y cualquier parámetro adicional necesario para su estrategia particular.

2. **Implementar `reset_internal_state`**: Establece o reinicia las variables internas que el agente utiliza para rastrear información durante su interacción con el entorno.

3. **Implementar `select_action`**: Define la lógica que el agente emplea para seleccionar una acción basada en la observación actual, la información proporcionada por el entorno y el estado interno del agente.

4. **Implementar `update_internal_state`**: Actualiza el estado interno del agente en función de la nueva información obtenida tras ejecutar una acción, como la nueva observación y la recompensa recibida.

5. **Implementar `get_extra_info`**: Proporciona información adicional sobre el estado o desempeño del agente, útil para análisis o depuración.


Preguntas que pueden ayudar a guiar la implementación de los agentes:
- ¿Qué información necesita el agente para tomar decisiones?
- ¿Cómo se actualiza el estado interno del agente después de cada interacción con el entorno?
- ¿Qué estrategias de selección de acciones basados en el conocimiento previo y la información actual se pueden utilizar?

**Evaluación del Desempeño de los Agentes**

Para analizar cómo se comportan nuestros agentes en el entorno, utilizaremos dos métricas clave:

1. **Recompensa Promedio por Paso:** Esta métrica nos muestra la recompensa media que un agente obtiene en cada paso a lo largo de múltiples episodios. Un valor más alto indica que el agente está tomando decisiones que le reportan mayores beneficios en promedio.

2. **Porcentaje de Selección de la Acción Óptima por Paso:** Esta métrica refleja la frecuencia con la que un agente selecciona la mejor acción disponible en cada paso. Un porcentaje elevado sugiere que el agente identifica y elige consistentemente la acción más beneficiosa.

Para obtener estas métricas, ejecutaremos cada agente durante un número determinado de episodios (`NUM_EPISODES`) y pasos por episodio (`NUM_STEPS`). Durante estas ejecuciones, registraremos las recompensas obtenidas y las acciones seleccionadas. Posteriormente, calcularemos el promedio de las recompensas y el porcentaje de veces que se seleccionó la acción óptima en cada paso.

Estas evaluaciones nos permitirán comparar el rendimiento de diferentes agentes y entender mejor sus comportamientos en el entorno propuesto. 

### Agente arbitrario (random)

Este agente selecciona una acción de manera aleatoria en cada paso, sin considerar ninguna información adicional. Es un agente de referencia simple que nos permitirá comparar el rendimiento de otros agentes más sofisticados.

In [ ]:
class RandomAgent(Agent):
    def __init__(self, name):
        super().__init__(name)        

    def reset_internal_state(self):
        pass

    def select_action(self, obs, info):
        pass
   
    def update_internal_state(self, observation, action, reward, info):
        pass
    
    def get_extra_info(self):
        pass

# Example run
random_agent = RandomAgent("RandomAgent")

logs, info = random_agent.play(n_steps = NUM_STEPS, environment = sample_env)

De este modo, podemos tener una estimación de la recompensa ($Q(a)$) de cada brazo.

In [ ]:
rewards_per_action = list(map(lambda x: np.array(x), logs['actions_rewards'].values())) # lista de recompensas por acción
labels = list(logs['actions_rewards'].keys()) # lista de etiquetas de las acciones

In [ ]:
plt.figure(figsize=(10, 5))
plt.boxplot(rewards_per_action, tick_labels=labels)
plt.title('Estimated Rewards per Action')
plt.show()

¿Coincide con el valor verdadero de cada brazo ($q_*(a)$)?

In [ ]:
plot_agents_performance([RandomAgent("RandomAgent")], num_steps=NUM_STEPS, num_episodes=NUM_EPISODES)

### Agente epsilon-greedy

El agentes epsilon-greedy es una estrategia simple que selecciona la mejor acción con probabilidad $1-\varepsilon$ y una acción aleatoria con probabilidad $\varepsilon$.

**Initialize, for** $a = 1$ to $k$:

$$
Q(a) \gets 0
$$

$$
N(a) \gets 0
$$

**Loop forever:**

$$
A \gets 
\begin{cases} 
\arg\max_a Q(a) & \text{with probability } 1 - \varepsilon \quad \text{(breaking ties randomly)} \\
\text{a random action} & \text{with probability } \varepsilon
\end{cases}
$$

$$
R \gets \text{bandit}(A)
$$

$$
N(A) \gets N(A) + 1
$$

$$
Q(A) \gets Q(A) + \frac{1}{N(A)} \left[ R - Q(A) \right]
$$



In [ ]:
class EpsilonGreedyAgent(Agent):
    def __init__(self, name, epsilon):
        super().__init__(name)        

    def reset_internal_state(self):
        pass

    def select_action(self, obs, info):
        pass
   
    def update_internal_state(self, observation, action, reward, info):
        pass
    
    def get_extra_info(self):
        pass


epsilon_greedy_agent = EpsilonGreedyAgent("EpsilonGreedyAgent", 0.1)

logs, info = epsilon_greedy_agent.play(n_steps = 1000, environment =sample_env)

print(info['n_a'])
print(info['q_a'])
print (f"Accumulated reward: {sum(logs['rewards']):.2f}")
print (f"Mean reward: {logs['rewards'].mean():.2f}")

In [ ]:
plot_agents_performance([
    EpsilonGreedyAgent("EpsilonGreedyAgent e=0.1", 0.1),
    EpsilonGreedyAgent("EpsilonGreedyAgent e=0.01", 0.01),
    EpsilonGreedyAgent("EpsilonGreedyAgent e=0", 0)
    ], num_steps=NUM_STEPS, num_episodes=NUM_EPISODES)

> Sugerencia:
> - Probar con otros valores de $\epsilon$ y comparar el rendimiento de los agentes. ¿Cómo afecta el valor de $\epsilon$ al rendimiento del agente?
> - ¿Qué ocurre si incrementamos la cantidad de pasos de entrenamiento? ¿Cómo afecta esto al rendimiento de los agentes?
>
>```python
>plot_agents_performance([
>    EpsilonGreedyAgent("EpsilonGreedyAgent e=a", a),
>    EpsilonGreedyAgent("EpsilonGreedyAgent e=b", b),
>    ...
>    EpsilonGreedyAgent("EpsilonGreedyAgent e=z", z)
>    ], num_steps=NUM_STEPS, num_episodes=NUM_EPISODES)
>```

### Agente epsilon-greedy optimista

El agente epsilon-greedy optimista es una variante del agente epsilon-greedy que inicializa los valores de recompensa estimados ($Q(a)$) con un valor optimista, en lugar de cero. Esto fomenta la exploración de acciones menos conocidas al principio, ya que el agente cree que todas las acciones tienen un valor más alto del que realmente tienen.

Preguntas:
- ¿Qué ocurre si ponemos un valor optimista muy alto?
- ¿Qué ocurre si ponemos un valor optimista muy bajo?

**Initialize, for** $a = 1$ to $k$:

$$
Q(a) \gets q_1
$$

$$
N(a) \gets 0
$$

**Loop forever:**

$$
A \gets 
\begin{cases} 
\arg\max_a Q(a) & \text{with probability } 1 - \varepsilon \quad \text{(breaking ties randomly)} \\
\text{a random action} & \text{with probability } \varepsilon
\end{cases}
$$

$$
R \gets \text{bandit}(A)
$$

$$
N(A) \gets N(A) + 1
$$

$$
Q(A) \gets Q(A) + \alpha (R - Q(A))
$$

> Nota: para esta implementación, usamos $alpha$ en lugar de $\frac{1}{N(A)}$ para actualizar el valor de $Q(A)$. Ver capítulo 2.5 del libro de Sutton y Barto.

In [ ]:
class OptimisticEpsilonGreedyAgent(Agent):
    def __init__(self, name, epsilon, q_1, alpha = 0.1):
        super().__init__(name)
        pass       

    def reset_internal_state(self):
        pass

    def select_action(self, obs, info):
        pass
   
    def update_internal_state(self, observation, action, reward, info):
        pass
    
    def get_extra_info(self):
        pass

# Example run
optimistic_epsilon_greedy_agent = OptimisticEpsilonGreedyAgent("OptimisticEpsilonGreedyAgent", epsilon = 0.1, q_1 = 0)

logs, info = optimistic_epsilon_greedy_agent.play(n_steps = 1000, environment = sample_env)
print(info['n_a'])
print(info['q_a'])
print (f"Accumulated reward: {sum(logs['rewards']):.2f}")
print (f"Mean reward: {logs['rewards'].mean():.2f}")

In [ ]:
plot_agents_performance([
    OptimisticEpsilonGreedyAgent("OptimisticEpsilonGreedyAgent e=0.1 Q1=0", epsilon = 0.1, q_1 = 0),
    OptimisticEpsilonGreedyAgent("OptimisticEpsilonGreedyAgent e=0   Q1=5", epsilon = 0,   q_1 = 5),
    ], num_steps=NUM_STEPS, num_episodes=NUM_EPISODES)

### Agente Upper Confidence Bound (UCB)

El agente UCB es una estrategia de selección de acciones que equilibra de forma inteligente la exploración y la explotación. En lugar de elegir acciones de forma aleatoria con cierta probabilidad (como en epsilon-greedy), el agente UCB asigna a cada acción una "bonificación de exploración" que disminuye conforme la acción es seleccionada más veces. Esta bonificación se combina con la estimación actual del valor $Q(a)$ para formar un criterio de selección.

Para cada acción $a$, se calcula:

$$
UCB(a) = Q(a) + c \sqrt{\frac{\ln t}{N(a)}}
$$

donde:
- $Q(a)$ es la estimación del valor esperado de la acción $a$.
- $N(a)$ es el número de veces que se ha seleccionado la acción $a$.
- $t$ es el número total de pasos (o iteraciones) realizados hasta el momento.
- $c$ es un parámetro que determina el grado de exploración: un valor mayor favorece la exploración de acciones menos probadas.

**Inicialización:**

Para cada acción $a = 1, \dots, k$:

$$
Q(a) \gets 0
$$
$$
N(a) \gets 0
$$
$$
t \gets 0
$$

**Algoritmo:**

**Loop forever:**

   $$
   t \gets t + 1
   $$

   $$
   A \gets \arg\max_a \left\{ Q(a) + c \sqrt{\frac{\ln t}{N(a)}} \right\}
   $$
   $$
   R \gets \text{bandit}(A)
   $$
   $$
   N(A) \gets N(A) + 1
   $$
   $$
   Q(A) \gets Q(A) + \frac{1}{N(A)} \left( R - Q(A) \right)
   $$
   
>Notas: 
>   - Para aquellas acciones que aún no han sido seleccionadas ($N(a) = 0$), se suele asignar un valor infinito al término de exploración para garantizar que cada acción se pruebe al menos una vez.
>   - Se puede usar una tasa de aprendizaje $\alpha$ para actualizar $Q(A)$ en lugar de $\frac{1}{N(A)}$.

### Preguntas:

- **¿Qué sucede si el parámetro $c$ es muy alto?**
- **¿Qué sucede si el parámetro $c$ es muy bajo?**  

In [ ]:
class UCBAgent(Agent):
    def __init__(self, name, C):
        super().__init__(name) 

    def reset_internal_state(self):
        pass

    def select_action(self, obs, info):
        pass
        
   
    def update_internal_state(self, observation, action, reward, info):
        pass
    
    def get_extra_info(self):
        pass

# Example run
UCB_agent = UCBAgent("UCBAgent", 1)

logs, info = UCB_agent.play(n_steps = 1000, environment =sample_env)
print(info['n_a'])
print(info['q_a'])
print (f"Accumulated reward: {sum(logs['rewards']):.2f}")
print (f"Mean reward: {logs['rewards'].mean():.2f}")

In [ ]:
plot_agents_performance([
    UCBAgent("UCB Agent C=2", 2),
    EpsilonGreedyAgent("EpsilonGreedyAgent e=0.1", 0.1),
    ], num_steps=NUM_STEPS, num_episodes=NUM_EPISODES)

## Tareas

1. Implementar los agentes mencionados.
2. Comentar los resultados obtenidos en las gráficas.
3. Seleccionar de cada agente el/los mejores hiperparámetros.
4. Tomar el mejor candidato de cada agente y compararlos en un mismo gráfico.
5. Explique porque en algunos agentes sus graficas tienen picos (epsilon-greedy optimista y UCB).
6. [Extra] Implementar un agente [epsilon-greedy con decay](https://arxiv.org/pdf/1910.13701).